In [2]:
import numpy as np
import pandas as pd


### Get Data

In [3]:
from sklearn import datasets
iris = datasets.load_iris()


In [4]:
columns = ['sepal_length', 'spal_width', 'petal_length', 'petal_width']
features = pd.DataFrame(iris.data, columns=columns)

In [5]:
target = pd.DataFrame(iris.target)

In [6]:
data = features.assign(type=target)

### Node Class

In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        '''                 Constructor                 '''
        # for decision node:
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        #for leaf:
        # Majority Class of the leaf node
        self.value = value

### Tree Class

In [ ]:
class DecisionTreeClassifier():
    ''' Constructor '''
    def __init__(self, min_sample_split=2, max_depth=2):
        #Initalise the root of the tree
        self.root = None
        # Define Stopping Conditions:
        self.max_depth = max_depth
        self.min_sample_split = min_sample_split
    ''' build tree method 
    
    - Recursively build the tree by finding the best split until stopping 
    conditions are met 
    Steps: 
    - Stopping Condition:
        if the number of samples during a split is less then the minimum 
        number of samples, or max depth of the tree is reached then return leaf 
        node (the prediction)
    - Best Split: 
        Calls best split to find the best feature and threhold to 
        split on
    - Create Leaf node: 
        if no valid split is found or stopping conditions are met, then return the leaf
        node
    '''
    def build_tree(self, dataset, current_depth=0):
        X = dataset.iloc[:,:-1]
        Y = dataset.iloc[:,-1]
        num_samples = np.shape(X)[0]
        num_features = np.shape(X)[1]
        
       
        
        
        if num_samples > self.min_sample_split & current_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_features)
            if best_split["info_gain"] > 0:
                left_subtree = self.build_tree(best_split["dataset_left"], current_depth + 1)
                right_subtree = self.build_tree(best_split["dataset_right"], current_depth + 1)
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
            
    
    
    
    
    ''' get_best_split 
    
    For a given data set containing features and target values:
    This function will for each feature will:
    - define an empty dictionary to hold node values
    - define max_information_gain as an infinitaly large number
    
        1. Get the feature values from dataset 
        2. get possible threhold values by finding the unique data points in each feature
        for each threshold value in possible_threshold values:
            1. get left and right datasets using the split function
            2. if the legt and right datasets are not null:
                3. list of classes in total (parent) list of classes in the left dataset (left_y) and list 
                of classes in the right data set (right_y)
                4. Calcualte information gain as current_information_gain
                5. if this is greater then the max_information gain:
                    6. create best split dictionary where you have each nodes value: 
                    - feature_index
                    - threshold
                    - dataset_left
                    - dataset_right
                    - info_gain
                7. Assign the max information gain as the current information gain
                8. Return the best split dictionary
    
    
    '''
    def get_best_split(self, dataset, num_features):
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset.iloc[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check the datasets are not empty:
                if len(dataset_left) > 0 & len(dataset_right)>0:
                    y, left_y, right_y = dataset.iloc[:,-1], dataset_left[:,-1], dataset_right[:,-1]
                    current_info_gain = self.info_gain(y, left_y, right_y, mode="entropy")
                    if current_info_gain > max_info_gain:
                        best_split = {
                            "feature_index" : feature_index,
                            "threshold" : threshold,
                            "dataset_left" : dataset_left,
                            "dataset_right" : dataset_right,
                            "info_gain" : current_info_gain
                        }
                        max_info_gain = current_info_gain
        return best_split
            
        
    
    
    ''' split 
    
    This function given a threshold value will split the data set of each feature
    and either assign it to the left subtree if the value is less then the threhold or 
    assign it to the right if the value is greater then the threshold

    '''
    def split(self, dataset, feature_index, threshold):
        left_dataset = np.array(row for row in dataset[feature_index] <= threshold)
        right_dataset = np.array(row for row in dataset[feature_index] > threshold)
        return left_dataset, right_dataset
    
    
    
    ''' Information gain 
    1. Calculates the weight of the left and the right childs 
    if mode is entropy : 
    gain = entropy(parent) - (left_weight * entropy(left_child) + right_weight * entropy(right_child))
    if mode is gini_index
    gain = gini_index(parent) - (left_weight * gini_index(left_child) + right_weight * gini_index(right_child))
    
    '''
    def info_gain(self, parent, left_child, right_child, mode="entropy"):
        weight_left = len(left_child) / len(parent)
        weight_right = len(right_child) / len(parent)
        if mode == "gini":
            gain = self.gini_index(parent) - ((weight_left * self.gini_index(left_child)) + (weight_right * self.gini_index(right_child)))
        else:
            gain = self.entropy(parent) - ((weight_left * self.entropy(left_child)) + (weight_right * self.entropy(right_child)))
        return gain
    
    ''' entropy 
    
    returns the entropy of a given class 
    1. identify the unique classes you have, in the iris data set its 0, 1, 2 (three types)
    2. For each unique class calculate entropy using the formula:
    H(p1) = -p1 * Log2(p1)
    Example: 
    y = np.array([0, 0, 1, 1, 1])
    •	Step 1: np.unique(y) returns the unique classes: [0, 1].
	•	Step 2: We initialize entropy = 0.
	•	Step 3: Loop through each class:
	•	For class 0:
	•	Proportion of class 0: p_0 = 2/5 = 0.4.
	•	Entropy term: -0.4 * log2(0.4) ≈ 0.52877.
	•	Add this to entropy: entropy ≈ 0.52877.
	•	For class 1:
	•	Proportion of class 1: p_1 = 3/5 = 0.6.
	•	Entropy term: -0.6 * log2(0.6) ≈ 0.44218.
	•	Add this to entropy: entropy ≈ 0.52877 + 0.44218 = 0.97095.
    
    '''
    def entropy(slef, y):
        # Find the unique classes in your target data set:
        classes = np.unique(y)
        # Initialise entropy to 0:
        entropy = 0
        for cls in classes:
            # get the porpotion :
            p_one = len(y[y == cls]) / len(y)
            # calculate entropy for this porportion and add to total entropy for this class
            entropy += -p_one * np.log2(p_one)
        return entropy
    
    def gini_index(self, y):
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
    
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    